[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andresfcm97/ia4eng_afcm_eac_proyecto/blob/main/01 - Exploración de Datos.ipynb)

In [ ]:
!pip install kaggle

In [ ]:
import google.colab as gc
gc.files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c house-prices-advanced-regression-techniques
from zipfile import ZipFile
file_name = 'house-prices-advanced-regression-techniques.zip'
with ZipFile(file_name, 'r') as zip:
  zip.extractall()

# 1. Importamos librerías que vamos a utilizar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from sklearn.model_selection import train_test_split

# 2. Cargamos los archivos que se van a utilizar

In [ ]:
entrenamiento = pd.read_csv("train.csv")
prueba = pd.read_csv("test.csv")
entrenamiento.head()

Nos informamos de las dimensiones de nuestros DataFrames

In [ ]:
ID_prueba = prueba['Id']
print(f"Dimensión train.csv: {entrenamiento.shape}\nDimensión test.csv: {prueba.shape}")

Ya que el DataFrame en el que más nos enfocaremos es en el de entrenamiento, es necesario informarse sobre si tiene o no algunas columnas con valores nulos, puesto que, después de verificar las variables que se tendrán presentes para el modelo, habrá que eliminar los datos nulos que contengan.

In [ ]:
entrenamiento.info()

In [ ]:
entrenamiento.columns

Tal como se pudo observar, el archivo train.csv cuenta con 1460 filas y 81 columnas, el objetivo será predecir el precio de venta de las casas, es decir, la columna "SalePrice", para esto, tendremos que mirar cuáles variables se relacionan con nuestra variable de interés (SalePrice), y elegir las que tengan mayor relación.